# Interactive Jupyter Notebooks for the visual analysis of critical choices in Global Sensitivity Analysis

### Valentina Noacco, Andres Peñuela-Fernandez, Francesca Pianosi, Thorsten Wagener 
### (University of Bristol)


[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)

This document provides:
* a brief introduction to Global Sensitivity Analysis (GSA);
* a workflow to asssess one of the critical choices needed to set up a GSA application. Here we use the SAFE (Sensitivity Analysis For Everybody) toolbox ([References](#References) 1-2).

In this example we apply the Regional Sensitivity Analysis GSA method to the rainfall-runoff Hymod model.


# PART I: Introduction

## What is Global Sensitivity Analysis and why shall we use it?

**Global Sensitivity Analysis** is a set of mathematical techniques which investigate how the uncertainty in the model inputs influences the variability of the model outputs ([References](#References) 3-4).

The benefits of applying GSA are:

* **Better understanding of the model**: Evaluate the model behaviour beyond default set-up
    
* **“Sanity check” of the model**: Test whether the model "behaves well" (model validation)
    
* **Priorities for uncertainty reduction**: Identify the important inputs on which to focus computationally-intensive calibration, acquisition of new data, etc. 
    
* **More transparent and robust decisions**: Understand how assumptions about uncertain inputs reflect on the modelling outcome and thus on model-informed decisions


## How Global Sensitivity Analysis works

GSA investigates how the uncertainty in the model input factors influences the variability of the model output.

An '**input factor**' is any element that can be changed before running the model. In general, input factors could be the model's parameters, forcing input data, but also the very equations implemented in the model or other set-up choices (for instance, the spatial resolution) needed for the model execution on a computer.

An '**output**' is any variable that is obtained after the model execution.

The main steps to perform GSA are summarised in the figure below.
<img src="how_GSA_works.png" width="800px">
a. The input factors are sampled from their ranges of variability.

b. The model is repeatedly run against each of the input sampled combinations.

c. The output samples so obtained can be used to characterise the output uncertainty, for instance through a probability distribution or scatter plots.

d. GSA is applied to the input and output samples in order to obtain a set of sensitivity indices. The sensitivity indices measure the relative influence of each input factor on the uncertainty of the output ([References](#References) 4,5).

# PART II: Workflow application
### Step 1: Import python modules

In [2]:
from __future__ import division, absolute_import, print_function

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from ipywidgets import widgets

# Import SAFE modules:
import os
mydir = "C:/Users/valen/OneDrive - University of Bristol/proj_SAFEVAL/SAFEPy/SAFEpython_v0.0.0"
os.chdir(mydir + "/SAFEpython")

import SAFEpython.RSA_groups as Rg # module to perform RSA with groups
import SAFEpython.plot_functions as pf # module to visualize the results
from SAFEpython.model_execution import model_execution # module to execute the model
from SAFEpython.sampling import AAT_sampling, AAT_sampling_extend # module to perform the input sampling
from SAFEpython.util import aggregate_boot  # function to aggregate the bootstrap results
from SAFEpython import HyMod


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/valen/OneDrive - University of Bristol/proj_SAFEVAL/SAFEPy/SAFEpython_v0.0.0/SAFEpython'

### Test model: Rainfall-runoff Hymod model

Before applying GSA, let's have a brief overview of the HyMod model, which it is a parsimonious rainfall-runoff model based on the theory of runoff yeild under infiltration excess. 

Hymod (Boyle 2001; Wagener et al. 2001) is composed of a soil moisture accounting routine, and a flow routing routine, which in its turn is composed of a fast and a slow routing pathway.

The model structure is illustrated in the figure below.

<left><img src="Hymod_scheme.png" width="700px">


### The following steps will be performed below:

### Step 2: Setup the model

Define: 
- the input factors whose influence is to be analysed with GSA, 
- their range of variability (choice made by expert judgement, available data or previous studies),
- choice of their distributions.

In [ ]:
M = 5 # number of input factors
N = 2000 # number of samples
distr_par  = [np.nan] * M
col_names  = ["$S_M$" , "$beta$" , "$alpha$", "$R_s$", "$R_F$"] # input factors of interest
distr_fun  = st.uniform # uniform distribution
samp_strat = 'lhs' # Latin Hypercube
fun_test   = HyMod.hymod_MulObj

In [ ]:
# range of variability
data = [["mm", 1,   400, "Maximum storage capacity"],
        ["-",  0,   2,   "Degree of spatial variability of the $S_M$"],
        ["-",  0,   1,   "Factor distributing slow and quick flows"],
        ["-",  0,   0.1, "Fractional discharge of the slow release reservoir"],
        ["-",  0.1, 1,   "Fractional discharge of the quick release reservoirs"]]
model_inputs = pd.DataFrame(data, 
                           columns=["Unit", "Min value", "Max value", "Description"],
                           index = col_names)
model_inputs

In [3]:
# Load data:
data = np.genfromtxt(mydir +'/data/LeafCatch.txt', comments='%')
rain = data[0:365, 0] # 1-year simulation
evap = data[0:365, 1]
flow = data[0:365, 2]
warmup = 30 # Model warmup period (days)

OSError: C:/Users/valen/OneDrive - University of Bristol/proj_SAFEVAL/SAFEPy/SAFEpython_v0.0.0/data/LeafCatch.txt not found.

In [6]:
class setup_model:
    def __init__(self, x1, x2, x3, x4, x5):
        # The shape parameters of the uniform distribution are the lower limit and 
        # the difference between lower and upper limits:
        self.xmin = [x1.value[0], x2.value[0], x3.value[0], x4.value[0], x5.value[0]]
        self.xmax = [x1.value[1], x2.value[1], x3.value[1], x4.value[1], x5.value[1]]
        for i in range(M):
            distr_par[i] = [self.xmin[i], self.xmax[i] - self.xmin[i]]
        self.distr_par = distr_par

### Step 3: Sample inputs space

The number of model runs (N) typically increases proportionally to the number of input factors ($M$) and will depend on the GSA methods chosen as well. 

In [7]:
class sample_input:
    def __init__(self,distr_par):
        self.X = AAT_sampling(samp_strat, M, distr_fun, distr_par, N)

### Step 4: Run the model

For each sampled input factors combination, we run the model and save the associated model output.

In [8]:
class run_model:
    def __init__(self,X):
        self.Y = model_execution(fun_test, X, rain, evap, flow, warmup)

### Step 5: Apply Sensitivity Analysis (RSA) method

RSA requires to sort the output samples and then to split them into a certain number of groups (defined by the user). Afterwards, RSA identifies the sub-samples in the inputs space that produced the outputs in each group and compute the cumulative distribution function (CDF) of each sub-sample. Finally, the sensitivity indices are defined as the (mean) maximum vertical distance between the CDFs of the various groups.


Here we divide the output samples into 10 groups, where each group contains the same number of samples.

In [9]:
class RSA_method:
    def __init__(self,X,Y): 
        mvd_median, mvd_mean, mvd_max, spread_median, spread_mean, spread_max, self.idx, self.Yk = \
        Rg.RSA_indices_groups(X, Y, ngroup=10, Nboot=0)

        self.mvd_p = pd.DataFrame(mvd_median,columns=["y"])
        Xdf = pd.DataFrame(X,columns=[col_names])
        xx = pd.melt(Xdf)
        Ydf = pd.DataFrame(Y,columns=["y"])
        yy = pd.concat([Ydf]*M,ignore_index=True)
        self.new_data = pd.concat([xx,yy],axis=1)
        self.new_data.columns = ["Input","x","y"]
        all_ind = np.concatenate((self.idx,self.idx,self.idx,self.idx,self.idx),axis=0)
        all_ind = pd.DataFrame(all_ind,columns=['idx'])
        self.new_data['idx'] = all_ind

### Step 6: Check model behaviour by visualising input/output samples

Scatterplots are plotted to visualise the behaviour of the output over each input factor in turn.

Definition of interactivity

In [10]:
def update_figures(change):
    with fig.batch_update():
        distr_par = setup_model(x1, x2, x3, x4, x5).distr_par
        X = sample_input(distr_par).X
        Y = run_model(X).Y[:,0]
        new_data = RSA_method(X,Y).new_data
        fig.data[0].x = new_data.loc[new_data['Input'] == model_inputs.index[0], 'x']    
        fig.data[1].x = new_data.loc[new_data['Input'] == model_inputs.index[1], 'x']    
        fig.data[2].x = new_data.loc[new_data['Input'] == model_inputs.index[2], 'x']
        fig.data[3].x = new_data.loc[new_data['Input'] == model_inputs.index[3], 'x']
        fig.data[4].x = new_data.loc[new_data['Input'] == model_inputs.index[4], 'x']
        fig.data[0].marker.color = new_data.loc[new_data['Input'] == model_inputs.index[0], 'idx']
        fig.data[1].marker.color = new_data.loc[new_data['Input'] == model_inputs.index[1], 'idx']    
        fig.data[2].marker.color = new_data.loc[new_data['Input'] == model_inputs.index[2], 'idx']
        fig.data[3].marker.color = new_data.loc[new_data['Input'] == model_inputs.index[3], 'idx']
        fig.data[4].marker.color = new_data.loc[new_data['Input'] == model_inputs.index[4], 'idx']
        fig.data[0].y = new_data.loc[new_data['Input'] == model_inputs.index[0], 'y']    
        fig.data[1].y = new_data.loc[new_data['Input'] == model_inputs.index[1], 'y']    
        fig.data[2].y = new_data.loc[new_data['Input'] == model_inputs.index[2], 'y']
        fig.data[3].y = new_data.loc[new_data['Input'] == model_inputs.index[3], 'y']
        fig.data[4].y = new_data.loc[new_data['Input'] == model_inputs.index[4], 'y']
        fig.layout.barmode = 'overlay'
        #fig.update_xaxes(matches=None)

        mvd_p = RSA_method(X,Y).mvd_p
        for i in range(0,5):
            fig1.data[i].y = np.array(mvd_p.y[i])

Definition of the sliders

In [11]:
x1 = widgets.FloatRangeSlider(value = [50, 150], min = 0, max = 200, step = 1, description = model_inputs.index[0], 
                              readout_format = '.1f', continuous_update=False)
x1.observe(update_figures,names = 'value')

x2 = widgets.FloatRangeSlider(value = [0.5, 1.5], min = 0, max = 2, step = 0.1, description = model_inputs.index[1], 
                              readout_format = '.1f', continuous_update=False)
x2.observe(update_figures,names = 'value')

x3 = widgets.FloatRangeSlider(value = [0.2, 0.8], min = 0, max = 1, step = 0.1, description = model_inputs.index[2], 
                              readout_format = '.1f', continuous_update=False)
x3.observe(update_figures,names = 'value')

x4 = widgets.FloatRangeSlider(value = [0.02, 0.08], min = 0, max = 0.1, step = 0.01, description = model_inputs.index[3], 
                              readout_format = '.1f', continuous_update=False)
x4.observe(update_figures,names = 'value')

x5 = widgets.FloatRangeSlider(value = [0.2, 0.8], min = 0, max = 1, step = 0.1, description = model_inputs.index[4], 
                              readout_format = '.1f', continuous_update=False)
x5.observe(update_figures,names = 'value')

Definition of the figure

In [12]:
distr_par = setup_model(x1, x2, x3, x4, x5).distr_par
X = sample_input(distr_par).X
Y = run_model(X).Y[:,0]
new_data = RSA_method(X,Y).new_data
p = px.scatter(new_data, x = "x", y = 'y', facet_col = "Input",color="idx", 
               color_continuous_scale = px.colors.diverging.RdYlBu[::-1],
               width=1000, height=500)
fig = go.FigureWidget(data=p, layout=go.Layout(barmode='overlay'))
fig.update_xaxes(title_text = "")
fig.update_xaxes(matches=None)
fig.layout.yaxis.title="Y"

c:\users\andro\onedrive - university of bristol\andresfrancesca\mat\jupyter notebooks\safe-notebooks\env\lib\site-packages\SAFEpython\util.py:24: NumbaWarning:


Compilation is falling back to object mode WITH looplifting enabled because Function "empiricalcdf" failed type inference due to: Untyped global name 'isinstance': cannot determine Numba type of <class 'builtin_function_or_method'>

File "env\lib\site-packages\SAFEpython\util.py", line 72:
def empiricalcdf(x, xi):
    <source elided>
    ###########################################################################
    if not isinstance(x, np.ndarray):
    ^


c:\users\andro\onedrive - university of bristol\andresfrancesca\mat\jupyter notebooks\safe-notebooks\env\lib\site-packages\SAFEpython\util.py:24: NumbaWarning:


Compilation is falling back to object mode WITHOUT looplifting enabled because Function "empiricalcdf" failed type inference due to: Untyped global name 'isinstance': cannot determine Numba type of <class 'builtin_

### Step 7: Plot sensitivity indices

The sensitivity indices for the RSA method are the maximum vertical distances over each pair of CDF.

Definition of the figure

In [13]:
%%capture
mvd_p = RSA_method(X,Y).mvd_p
p1 = px.scatter(mvd_p, x = ["$S_M$" , "$beta$" , "$alpha$", "$R_s$", "$R_F$"], y = 'y',
                color=['red','blue','green',"magenta","cyan"], width=600, height=300)

fig1  = go.FigureWidget(data=p1,layout=go.Layout(barmode='overlay'))
fig1.layout.yaxis.range=[0,1]
fig1.layout.yaxis.title="Maximum Vertical Distance"
fig1.layout.xaxis.title=""
fig1.update_traces(marker=dict(size=20),showlegend=False)

#### Plot the interactive figures + sliders

In [14]:
widgets.VBox([widgets.HBox([widgets.VBox([x1,x2,x3,x4,x5]),fig1]), fig])

### Step 8: Assess robustness by bootstrapping

In order to assess the robustness of the estimated sensitivity indices, bootstrapping is performed (here we resample 100 times). The 95% confidence intervals of the sensitivity indices are plotted below.

In [15]:
# Compute sensitivity indices with confidence intervals using bootstrapping
Nboot = 100
# Warning: the following line may take some time to run, as the computation of
# CDFs is costly:
mvd_median, mvd_mean, mvd_max, spread_median, spread_mean, spread_max, idx, Yk = \
    Rg.RSA_indices_groups(X, Y, ngroup=10, Nboot=100)

# Compute mean and confidence intervals of the sensitivity indices (mvd,
# maximum vertical distance) across the bootstrap resamples:
mvd_m, mvd_lb, mvd_ub = aggregate_boot(mvd_median) # shape (M,)

mvd_m_p = pd.DataFrame(mvd_m,columns=["y"])
mvd_lb_p = pd.DataFrame(mvd_lb,columns=["y"])
mvd_ub_p = pd.DataFrame(mvd_ub,columns=["y"])

fig2 = px.scatter(mvd_m_p, x = ["x1","x2", "x3"], y = 'y',color=['black','black','black'], width=600, height=400)
fig2.add_scatter(y = mvd_lb_p.y, x = ["x1","x2", "x3"], marker=dict(color=['red','blue','green']), mode = "markers")
fig2.add_scatter(y = mvd_ub_p.y, x = ["x1","x2", "x3"], marker=dict(color=['red','blue','green']), mode = "markers")

fig2.layout.yaxis.range=[0,1]
fig2.layout.yaxis.title="Maximum Vertical Distance"
fig2.layout.xaxis.title=""
fig2.update_traces(marker=dict(size=16),showlegend=False)
fig2.show()

ValueError: All arguments should have the same length. The length of column argument `df[y]` is 5, whereas the length of previous arguments ['x'] is 3

### Step 9: Visualise input factors interactions

In order to investigate the interactions between input factors we plot one input against the other, coloured by the value taken by the output.

In [ ]:
data_inter = pd.concat([pd.DataFrame(X),pd.DataFrame(Y)],axis=1)
data_inter.columns = ['x1', 'x2', 'x3', 'y']
# index_vals = data_colscat['idx'].astype('category').cat.codes

In [ ]:
fig3 = go.Figure(data=go.Splom(
                dimensions=[dict(label='x1',
                                 values=data_inter['x1']),
                            dict(label='x2',
                                 values=data_inter['x2']),
                            dict(label='x3',
                                 values=data_inter['x3'])],
                showupperhalf=False, # remove plots on diagonal
                text=data_inter['y'],
                marker=dict(color=data_inter['y'],colorbar=dict(title="Y"),
                            showscale=True, # colors encode categorical variables
                            line_color='white', line_width=0.5,colorscale = px.colors.diverging.RdYlBu[::-1])
                ))

fig3.show()                       

Here we visualise interactions between inputs through parallel coordinate plots.

In [ ]:
data_group = pd.concat([pd.DataFrame(X),pd.DataFrame(Y),pd.DataFrame(idx)],axis=1)
data_group.columns = ['x1', 'x2', 'x3', 'y','idx']

fig4 = px.parallel_coordinates(data_group, color="y", dimensions=['x1','x2','x3','y'],
                             color_continuous_scale=px.colors.diverging.RdYlBu[::-1],
                             color_continuous_midpoint=2)
fig4.show()

### <a id="References"></a>References


RSA is based on the function created as part of the SAFE Toolbox by F. Pianosi, F. Sarrazin and T. Wagener at Bristol University (2015). Please refer to the `Licence` file in the SAFE toolbox. 

1) [SAFE Toolbox Website](https://www.safetoolbox.info/)	

2) [Introductory paper to SAFE - Pianosi et al. (2015)](https://www.sciencedirect.com/science/article/pii/S1364815215001188)

3) [A review of available methods and workflows for Sensitivity Analysis - Pianosi et al. (2016)](https://www.sciencedirect.com/science/article/pii/S1364815216300287)

4) [What has Global Sensitivity Analysis ever done for us? A systematic review to support scientific advancement and to inform policy-making in earth system modelling - Wagener and Pianosi (2019)](https://www.sciencedirect.com/science/article/pii/S0012825218300990)

5) [Practical guide through the critical choices needed for Global Sensitivity Analysis - Noacco et al. (2019)](https://www.sciencedirect.com/science/article/pii/S2215016119302572?via%3Dihub)